In [1]:
from layout_module import Layout, NestedTuple
from layout_module import layout_algebra as la
from layout_module import flat_algebra as fa 
import numpy as np

In [2]:
lane_shape = (4,4)
lane_stride = (1,4)
load_shape = (1,8)
load_stride = (0,1)
lane_layout = Layout(NestedTuple.from_literal(lane_shape), NestedTuple.from_literal(lane_stride))
load_layout = Layout(NestedTuple.from_literal(load_shape), NestedTuple.from_literal(load_stride))
loader = la.product(load_layout, lane_layout)

In [3]:
shared_memory_shape = NestedTuple.from_literal((16,8))
shared_memory_stride = NestedTuple.from_literal((8,1))
smem_layout = Layout(shared_memory_shape, shared_memory_stride)

access_layout = la.compose(smem_layout, loader)
access_layout.coalesce()

Layout(shape=(16, 8), stride=(8, 1))

In [4]:
warp_layout = Layout(NestedTuple.from_literal((32,2,2)), NestedTuple.from_literal((4,2,1)))

In [5]:
L = 32
R_A = 4 
P_A = 2 
mma_m = 16 
mma_k = 16

warp_A_frag_shape = (L,R_A,P_A)
warp_A_logical_shape = (mma_m,mma_k)
warp_A_frag_stride = (8,2,1)

warp_A_logical_stride = (mma_k,1)
logical_array = np.empty((mma_m, mma_k), dtype=object)

  


In [6]:
logical_layout_A = Layout(NestedTuple.from_literal(warp_A_logical_shape), NestedTuple.from_literal(warp_A_logical_stride))

wA_frag_layout = Layout(NestedTuple.from_literal(warp_A_frag_shape), NestedTuple.from_literal(warp_A_frag_stride))

In [7]:
logical_layout_A

Layout(shape=(16, 16), stride=(16, 1))

In [8]:
tiler_shape = NestedTuple.from_literal((8,8))
tiler_stride = NestedTuple.from_literal((8,1))

tiler_layout = Layout(tiler_shape,tiler_stride)

tiled_layout = la.divide(logical_layout_A, tiler_layout)

In [9]:
small_layout = Layout(NestedTuple.from_literal((2,2)), NestedTuple.from_literal((1,2)))

full_layout = la.product(small_layout, logical_layout_A)


In [10]:
full_layout

Layout(shape=((2, 2), (16, 16)), stride=((1, 2), (64, 4)))

In [11]:
G = la.compose(wA_frag_layout, full_layout)

In [12]:
G.relative_coalesce(wA_frag_layout.shape)

Layout(shape=((8, 4), (2, 2), (2,)), stride=((128, 4), (2, 64), (1,)))